## <span style="color: skyblue; ">7.3 コールバック応用</span>

- 画面遷移
- 各レイアウトごとにコールバックが存在するアプリケーション
- グラフ上のマウス動作の活用
- 特定状態の更新停止
- 連鎖コールバック

### <span style="color: skyblue; ">7.3.1 画面遷移</span>

- iris データセット
- アクセスできる URL（パス）を複数用意して、
- URL のリンクをクリックするとコンテンツが切り替わる（画面遷移）アプリケーション
- [Multi-Page Apps and URL Support](https://dash.plotly.com/urls)
  - [dcc.Location](https://dash.plotly.com/dash-core-components/location)
  - [dcc.Link](https://dash.plotly.com/dash-core-components/link)

In [1]:
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
from dash import html, dcc
from dash.dependencies import Input, Output

iris = px.data.iris()

app = JupyterDash(__name__)

# レイアウト
def server_layout():
    return html.Div([
        # URLの生成
        dcc.Location(id="my-location"),
        html.Div(
            id="show-location",
            style=dict(fontSize="30px", textAlign="center", height="400px")
        ),
        html.Br(),  # <br>
        # Linkの設置
        dcc.Link("home", href="/"),
        html.Br(),
        dcc.Link("/graph", href="/graph"),
        html.Br(),
        dcc.Link("/table", href="/table"),
    ], style=dict(fontSize="30px", textAlign="center"))

app.layout = server_layout

# ページごとのコンテンツの作成
# home(/)のコンテンツ
home = html.H1("irisデータ")

# graph(/graph)のコンテンツ
graph = dcc.Graph(
    figure=px.scatter(
        iris,
        x="sepal_width",
        y="sepal_length",
        color="species",
        title="Irisグラフ"
    )
)

# table(/table)のコンテンツ
table = dcc.Graph(
    figure=go.Figure(
        data=go.Table(
            header=dict(values=iris.columns),
            cells=dict(values=[iris[col].tolist() for col in iris.columns])
        ),
        layout=go.Layout(title="irisデータテーブル"),
    )
)

# 各pathnameごとに返すコンテンツを指定する
@app.callback(
    Output(component_id="show-location", component_property="children"),
    Input(component_id="my-location", component_property="pathname"),
)
def update_location(pathname):
    if pathname == "/graph":
        return graph
    elif pathname == "/table":
        return table
    else:
        return home
    
    
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


### <span style="color: skyblue; ">7.3.2 各レイアウトごとにコールバックが存在するアプリケーション</span>

- `JupyterDash(suppress_callback_exceptions=True)`
  - Dash では、起動時にコールバック用いられる ID とレイアウトに存在する ID を確認して、
  - 一致しない場合は例外を送出する仕様になっている。
  - この例外を抑制する。（`suppress_callback_exceptions=True`）

In [1]:
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
from dash import html, dcc
from dash.dependencies import Input, Output

iris = px.data.iris()

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = JupyterDash(
    __name__,
    external_stylesheets=external_stylesheets,
    suppress_callback_exceptions=True
)

# レイアウト
def server_layout():
    return html.Div(
        [
            dcc.Location(id="url"),
            html.Div(id="page-content1", style=dict(height="600px")),
            html.Br(),
            dcc.Link("/graph", href="/graph"),
            html.Br(),
            dcc.Link("/table", href="/table"),
        ],
        style=dict(textAlign="center")
    )
app.layout = server_layout

# ページごとのコンテンツ
# Home page
home = html.H1("irisデータ")

# Graph page
graph = html.Div([
    html.Div([
        html.Div([
            html.P("X軸: "),
            dcc.RadioItems(
                id="x-axis-radio",
                options=[
                    dict(label=col, value=col) for col in iris.columns[:4]
                ],
                value="sepal_width"
            )
        ]),
        html.Div([
            html.P("Y軸: "),
            dcc.RadioItems(
                id="y-axis-radio",
                options=[
                    dict(label=col, value=col) for col in iris.columns[:4]
                ],
                value="sepal_length"
            )
        ]),
        
    ], style=dict(display="flex")),
    dcc.Graph(id="radio-graph")
])

# Table page
table = html.Div([
    html.Div([
        dcc.Dropdown(
            id="species-dropdown",
            options=[dict(label=col, value=col) for col in iris.columns],
            multi=True,
            value=["sepal_length", "sepal_width"]
        )
    ], style=dict(width="60%", margin="auto")),
    dcc.Graph(id="table")
])

# ページ切替用コールバック
@app.callback(
    Output(component_id="page-content1", component_property="children"),
    Input(component_id="url", component_property="pathname"),
)
def display_page(pathname):
    if pathname == "/graph":
        return graph
    elif pathname == "/table":
        return table
    else:
        return home
    
    
# グラフ更新用コールバック
@app.callback(
    Output(component_id="radio-graph", component_property="figure"),
    Input(component_id="x-axis-radio", component_property="value"),
    Input(component_id="y-axis-radio", component_property="value"),
)
def update_graph(selected_x, selected_y):
    return px.scatter(
        iris,
        x=selected_x,
        y=selected_y,
        color="species",
        marginal_x="box",
        marginal_y="violin",
        title="iris グラフ"
    )
    
    
# テーブル更新用コールバック
@app.callback(
    Output(component_id="table", component_property="figure"),
    Input(component_id="species-dropdown", component_property="value")
)
def update_table(selected_value):
    iris_df = iris[selected_value]
    return go.Figure(
        data=go.Table(
            header=dict(values=iris_df.columns),
            cells=dict(values=[iris_df[col].tolist() for col in iris_df.columns])
        ),
        layout=go.Layout(title="iris データテーブル")
    )
    
    
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


### <span style="color: skyblue; ">7.3.3 グラフ上のマウス動作の活用</span>

- Graph コンポーネントの属性から、マウス操作でグラフ上の要素を得られる。

マウスアクションの種類

|引数|使い方|
|:-|:-|
|hoverData|グラフ上の1つの要素をホバーで取得する|
|clickData|グラフ上の1つの要素をクリックで取得する|
|relayoutData|グラフの指定した範囲の位置データを取得する|
|selectedData|グラフ上の複数要素を「右クリック + Shift」またはドラッグで取得する|

In [1]:
# hoverData属性を用いて、
# マウスカーソルを当てた散布図の要素のデータを表示する

import json
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
from dash import html, dcc
from dash.dependencies import Input, Output

# データ作成
gapminder = px.data.gapminder()
gapminder_2007 = gapminder.loc[gapminder["year"] == 2007]

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
style=dict(width="80%", margin="auto", textAlign="center")
def server_layout():
    return html.Div([
        html.H1("Gapminder Graph"),
        dcc.Graph(
            id="gapminder-graph",
            figure=px.scatter(
                gapminder_2007, x="gdpPercap", y="lifeExp", hover_name="country"
            )
        ),
        # ホバーデータを表示するP要素
        html.P(
            id="hover-data-p",
            style=dict(fontSize="32px", textAlign="center")
        )
    ], style=style)
app.layout = server_layout

# Callback
@app.callback(
    Output(component_id="hover-data-p", component_property="children"),
    Input(component_id="gapminder-graph", component_property="hoverData")
)
def show_hover_data(hoverData):
    return json.dumps(hoverData)

if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [1]:
# selectedData 属性を用いて、
# 散布図上から「右クリック + Shift」またはドラッグで選択した複数のデータを表示する
""" グラフから得られるデータ(json):
    {
        "points": [{
            "curveNumber": 0,
            "pointNumber": 7,
            "pointIndex": 7,
            "x": 29796.04834,
            "y": 75.635,
            "hovertext": "Bahrain"
        }]
    }
"""

import json
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
from dash import html, dcc
from dash.dependencies import Input, Output

# データ作成
gapminder = px.data.gapminder()
gapminder_2007 = gapminder.loc[gapminder["year"] == 2007]

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
style=dict(width="80%", margin="auto", textAlign="center")
def server_layout():
    return html.Div([
        html.H1("Gapminder Graph"),
        dcc.Graph(
            id="gapminder-graph",
            figure=px.scatter(
                gapminder_2007, x="gdpPercap", y="lifeExp", hover_name="country",
                # 右クリック + Shift」またはドラッグで複数のデータを選択
                template=dict(layout=dict(clickmode="event+select"))
            )
        ),
        # ホバーデータを表示するP要素
        html.P(
            id="hover-data-p",
            style=dict(fontSize="32px", textAlign="center")
        )
    ], style=style)
app.layout = server_layout

# Callback
@app.callback(
    Output(component_id="hover-data-p", component_property="children"),
    Input(component_id="gapminder-graph", component_property="selectedData")
)
def show_hover_data(selectedData):
    return json.dumps(selectedData)

if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


- https://plot.ly/python/reference/#layout-dragmode

In [1]:
# px.scatter 関数の引数 template の"dragmode"属性に"select"を渡して、
# 複数要素を選択する
# dragmode="zoom" (default)
# 複数の国を抽出して、
# 人口と平均寿命を折れ線グラフで描画する
""" グラフから得られるデータ(json):
    {
        "points": [{
            "curveNumber": 0,
            "pointNumber": 7,
            "pointIndex": 7,
            "x": 29796.04834,
            "y": 75.635,
            "hovertext": "Bahrain"
        }]
    }

"""

import json
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import html, dcc
from dash.dependencies import Input, Output

# データ作成
gapminder = px.data.gapminder()
gapminder_2007 = gapminder.loc[gapminder["year"] == 2007]

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
style=dict(width="80%", margin="auto", textAlign="center")
def server_layout():
    return html.Div([
        html.H1("Gapminder Graph"),
        dcc.Graph(
            id="gapminder-graph",
            figure=px.scatter(
                gapminder_2007, x="gdpPercap", y="lifeExp", hover_name="country",
                # 右クリック + Shift」またはドラッグで複数のデータを選択
                template=dict(layout=dict(dragmode="select"))
            )
        ),
        # ホバーデータを表示するP要素
        html.Div([
            dcc.Graph(id="line-chart1", className="six columns"),
            dcc.Graph(id="line-chart2", className="six columns")
        ])
    ], style=style)
app.layout = server_layout

# Callback
@app.callback(
    Output(component_id="line-chart1", component_property="figure"),
    Output(component_id="line-chart2", component_property="figure"),
    Input(component_id="gapminder-graph", component_property="selectedData")
)
def show_hover_data(selectedData):
    if selectedData:
        selected_countries = [data["hovertext"] for data in selectedData["points"]]
        selected_df = gapminder[gapminder["country"].isin(selected_countries)]
        fig1 = px.line(selected_df, x="year", y="pop", color="country", title="各国の人口")
        fig2 = px.line(selected_df, x="year", y="lifeExp", color="country", title="各国の平均寿命")
        return fig1, fig2
    raise dash.exceptions.PreventUpdate

if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


### <span style="color: skyblue; ">7.3.4 特定の状態でのコールバックの更新停止</span>

- 特定の状態でコールバックを更新しない場合
  - PreventUpdate クラス: コールバック全体を止める
  - no_update クラス: 部分的に出力が更新されない

In [1]:
# selectedData 属性を用いて、
# 散布図上から「右クリック + Shift」またはドラッグで選択した複数のデータを表示する

# アプリケーション起動直後に表示される「null」を停止する
# （ホバーデータが存在しないため、「null」と表示される）
# ホバーデータが存在しなければ、コールバック全体の「更新を停止するコールバック」を加える
# PreventUpdate を用いる
""" グラフから得られるデータ(json):
    {
        "points": [{
            "curveNumber": 0,
            "pointNumber": 7,
            "pointIndex": 7,
            "x": 29796.04834,
            "y": 75.635,
            "hovertext": "Bahrain"
        }]
    }
"""

import json
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import html, dcc
from dash.dependencies import Input, Output

# データ作成
gapminder = px.data.gapminder()
gapminder_2007 = gapminder.loc[gapminder["year"] == 2007]

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
style=dict(width="80%", margin="auto", textAlign="center")
def server_layout():
    return html.Div([
        html.H1("Gapminder Graph"),
        dcc.Graph(
            id="gapminder-graph",
            figure=px.scatter(
                gapminder_2007, x="gdpPercap", y="lifeExp", hover_name="country",
            )
        ),
        # ホバーデータを表示するP要素
        html.P(
            id="hover-data-p",
            style=dict(
                fontSize= "24px",
                textAlign= "center",
                height= "200px",
                backgroundColor= "#e1eef6",
            ),
        ),
        # コールバックの出力先
        html.P(
            id="prevent-update",
            style=dict(
                fontSize= "24px",
                textAlign= "center",
                height= "200px",
                backgroundColor= "#D7FFF1",
            ),
        ),
    ], style=style)
app.layout = server_layout

@app.callback(
    Output(component_id="hover-data-p", component_property="children"),
    Input(component_id="gapminder-graph", component_property="hoverData")
)
def show_hover_data(hoverData):
    return json.dumps(hoverData)


# PreventUpdate を用いたコールバック
@app.callback(
    Output(component_id="prevent-update", component_property="children"),
    Input(component_id="gapminder-graph", component_property="hoverData")
)
def prevent_none(hoverData):
    if hoverData is None:
        # PreventUpdate クラスを用いて更新を停止
        raise dash.exceptions.PreventUpdate
    return json.dumps(hoverData)


if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [1]:
# no_update クラスを用いて、
# 部分的にコールバックの更新を停止する
# 上記のコードで2つに分けたコールバックを1つにまとめる

import json
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import html, dcc
from dash.dependencies import Input, Output

# データ作成
gapminder = px.data.gapminder()
gapminder_2007 = gapminder.loc[gapminder["year"] == 2007]

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
style=dict(width="80%", margin="auto", textAlign="center")
def server_layout():
    return html.Div([
        html.H1("Gapminder Graph"),
        dcc.Graph(
            id="gapminder-graph",
            figure=px.scatter(
                gapminder_2007, x="gdpPercap", y="lifeExp", hover_name="country",
            )
        ),
        # ホバーデータを表示するP要素
        html.P(
            id="hover-data-p",
            style=dict(
                fontSize= "24px",
                textAlign= "center",
                height= "200px",
                backgroundColor= "#e1eef6",
            ),
        ),
        # コールバックの出力先
        html.P(
            id="prevent-update",
            style=dict(
                fontSize= "24px",
                textAlign= "center",
                height= "200px",
                backgroundColor= "#D7FFF1",
            ),
        ),
    ], style=style)
app.layout = server_layout

@app.callback(
    # データの状態に関係なくコールバックを更新する出力先
    Output(component_id="hover-data-p", component_property="children"),
    # データが None であれば、コールバックの更新を停止する出力先
    Output(component_id="prevent-update", component_property="children"),
    Input(component_id="gapminder-graph", component_property="hoverData")
)
def show_hover_data(hoverData):
    if hoverData is None:
        return (json.dumps(hoverData), dash.no_update)
    # 1つ目の戻り値はホバーデータをそのまま、2つ目の戻り値は更新を停止
    return json.dumps(hoverData),json.dumps(hoverData)


if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


### <span style="color: skyblue; ">7.3.5 連鎖コールバック</span>

- 連鎖コールバック
  - コールバックの出力を別のコールバックの入力に用いるコールバック
  - コールバックを連鎖させることにより、より複雑な動作が可能になる


In [1]:
# tips データセットを用いて、
# ドロップダウンでグラフの種類が選択でき、
# 選択時に更新されるラジオボタンでグラフの表示データが更新できる
# (1)Dropdown コンポーネントでグラフの種類を選択すると、RadioItems コンポーネントの選択肢がグラフに対応したものに更新される
# (2)各コンポーネントで選択された値を用いて figure が更新される
#
# (1)のコールバックでは、Dropdown コンポーネントの value 属性の変化により、
# コールバックが呼び出されて、RadioItems コンポーネントの選択肢などを戻り値とする
# (2)のコールバックでは、RadioItems コンポーネントの value 属性の変化をきっかけに呼び出され、figure を戻す

from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State

# データ作成
tips = px.data.tips()

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
def server_layout():
    layout = html.Div([
        # アプリケーションのタイトル
        html.H1(id="h1-title"),
        html.Div([
            html.Div([
                # グラフの種類を選択する Dropdown とタイトル
                html.P("グラフの種類の選択"),
                dcc.Dropdown(
                    id="drop-select-graph",
                    options=[
                        dict(label="棒グラフ", value="bar"),
                        dict(label="散布図", value="scatter"),
                    ],
                    value="bar"
                ),
                html.Div([
                    # グラフの表示要素を選択する RadioItems
                    html.P(id="p-graph-title"),
                    dcc.RadioItems(id="radio-options")
                ])
            ], style=dict(width="35%")),
            html.Div([
                # 選択された種類のグラフを表示する Graph
                dcc.Graph(id="show-graph")
            ], style=dict(width="65%", height="800px"))
        ], style=dict(display="flex"))
    ])
    return layout
app.layout = server_layout

# callback 1
# 表示項目の更新
@app.callback(
    Output(component_id="h1-title", component_property="children"),
    Output(component_id="p-graph-title", component_property="children"),
    Output(component_id="radio-options", component_property="value"),
    Output(component_id="radio-options", component_property="options"),
    Input(component_id="drop-select-graph", component_property="value")
)
def update_values(graph_type):
    if graph_type == "bar":
        return (
            "チップデータ（棒グラフ）",
            "棒グラフの選択肢",
            "total_bill",
            [
                dict(value="total_bill", label="総額"),
                dict(value="sex", label="性別"),
                dict(value="smoker", label="喫煙 / 禁煙"),
                dict(value="time", label="時間帯（昼 / 夜）"),
            ]
        )
    return (
        "チップデータ（散布図）",
        "散布図の選択肢",
        "smoker",
        [
            dict(value="smoker", label="喫煙 / 禁煙"),
            dict(value="sex", label="性別"),
            dict(value="day", label="曜日"),
            dict(value="time", label="時間帯（昼 / 夜）"),
        ]
    )


# callback 2
# グラフの更新
@app.callback(
    Output(component_id="show-graph", component_property="figure"),
    Input(component_id="radio-options", component_property="value"),
    State(component_id="drop-select-graph", component_property="value")
)
def update_graph(selected_value, graph_type):
    if graph_type == "bar":
        return px.bar(
            tips,
            x="day",
            y="total_bill",
            color=selected_value,
            barmode="group",
            height=600,
            title=f"チップデータ棒グラフ（要素: {selected_value}）"
        )
    else:
        return px.scatter(
            tips,
            x="total_bill",
            y="tip",
            color=selected_value,
            height=600,
            title=f"チップデータ散布図（色: {selected_value}）"
        )


if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
